### Import relevant packages, establish connection to WRDS and set overall configurations for the notebook

WRDS Support - https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/querying-wrds-data-python/

In [ ]:
# Import packages
import os
import numpy as np
import pandas as pd
import wrds
import yfinance as yf 

# Build WRDS connection

db = wrds.Connection(wrds_username='tomasromeiro')
#db.close()

# Set option to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set option to force dataframes to display numbers as floats with thousands separators
pd.set_option('display.float_format', '{:,.2f}'.format)  # Adjust decimal places as needed

Loading library list...
Done


WRDS Quick commands

In [ ]:
# List libraries available
sorted(db.list_libraries()) 

# List tables within a library
db.list_tables(library="cboe") 

# describe table metadata
db.describe_table(library="cboe", table="optprice_2024") 

# Execute a sql query against a table (join queries between tables in library can also be performed)
data = db.raw_sql('SELECT date, dji FROM djones.djdaily LIMIT 1', date_cols=['date']) 

# Pass parameters to a sql statement
params = {"tickers": ("0015B", "0030B", "0032A", "0033A", "0038A")}
data = db.raw_sql(
    "SELECT datadate, gvkey, cusip FROM comp.funda WHERE tic IN %(tickers)s LIMIT 1",
    params=params,
)

### 1. FINRA Short Interest Bimonthly Data  
- https://www.finra.org/finra-data/browse-catalog/equity-short-interest/files
- https://www.finra.org/finra-data/browse-catalog/equity-short-interest/glossary

#### a) Collate semi monthly datasets
If the data has already been collated do not run this and skip to b)

In [ ]:
# Define the directory containing the CSV files
directory = 'data/finra_short_interest_data'
output_file = os.path.join(directory, 'collated_short_interest_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Get a list of all pipe-delimited CSV files in the directory
csv_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.csv')]

# Read and concatenate all CSV files with proper delimiter handling
df_list = []
for file in csv_files:
    try:
        df = pd.read_csv(file, sep='|')  # Read as pipe-delimited with specific dtype
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if df_list:
    short_interest_df = pd.concat(df_list, ignore_index=True)
    
    # Replace daysToCoverQuantity with blank where averageDailyVolumeQuantity is 0
    short_interest_df.loc[short_interest_df['averageDailyVolumeQuantity'] == 0, 'daysToCoverQuantity'] = None
    
    # Remove all entries where the ticker (symbolCode) is missing and daysToCoverQuantity is NaN or 999.99 
    short_interest_df = short_interest_df.dropna(subset=['symbolCode', 'daysToCoverQuantity'])
    short_interest_df = short_interest_df[short_interest_df['daysToCoverQuantity'] != 999.99]

    # Keep only stocks not traded Over the Counter
    short_interest_df = short_interest_df[short_interest_df['marketClassCode'] != 'OTC']

    # Drop unnecessary fields
    short_interest_df = short_interest_df.drop(columns=['accountingYearMonthNumber', 'issuerServicesGroupExchangeCode', 'stockSplitFlag', 'revisionFlag', 'changePercent', 'changePreviousNumber', 'previousShortPositionQuantity', 'issueName', 'marketClassCode'])
    
    # Move settlementDate to the first column
    columns = ['settlementDate'] + [col for col in short_interest_df.columns if col != 'settlementDate']
    short_interest_df = short_interest_df[columns]

    # Sort by settlementDate and symbolCode
    short_interest_df = short_interest_df.sort_values(by=['settlementDate', 'symbolCode'])

    # Renaming columns
    short_interest_df.rename(columns={"currentShortPositionQuantity": "short_volume"}, inplace=True)
    short_interest_df.rename(columns={"averageDailyVolumeQuantity": "avg_daily_volume"}, inplace=True)
    short_interest_df.rename(columns={"daysToCoverQuantity": "days_to_cover"}, inplace=True)

    # Chaging fields to appropriate data type
    short_interest_df = short_interest_df.astype({"short_volume": "int32", "avg_daily_volume": "int32"})

    # Save the collated DataFrame to the same directory
    short_interest_df.to_csv(output_file, index=False)
    
    print(f"Collated data saved to {output_file}")
else:
    print("No valid CSV files found.")

#### b) Open .csv file to memory

In [24]:
# Define the directory containing the CSV files
directory = 'data/finra_short_interest_data'

# Open the collated file in a DataFrame for viewing
short_interest_file = os.path.join(directory, 'collated_short_interest_data.csv')
short_interest_df = pd.read_csv(short_interest_file)   

Extract tickers and date ranges to use as parameters for remaining data extracts

In [25]:
# Extract unique tickers from the short interest file. Will be used as the main variable to pass through to subsquent queries where tickers are required
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"{len(tickers)} unique tickers in the short interest file")

# Extract earliest and latest date in the short interest file
earliest_date = short_interest_df['settlementDate'].min()
latest_date = short_interest_df['settlementDate'].max()

print(f"Short interest file date range is {earliest_date} to {latest_date}")

16857 unique tickers in the short interest file
Short interest file date range is 2021-06-15 to 2025-01-15


### 2. WRDS (Wharton) Data

#### i) Company Data (Quarterly) - Fundamentals
https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/

We'll extract quarterly financial statement data and derive commonly used metrics if not available directly.

Variable references (for the quarterly reporting period, in USD):
- conm: company name
- tic: company ticker symbol
- rdq: report date of quarterly earnings
- revtq: total revenue 
- cogsq: cost of goods sold
- oiadpq: operating income after depreciation and amortisation
- dlcq: short-term (current) debt
- dlttq: long-term debt
- cheq: cash and cash equivalents at reporting point in time

The variables above will be used to calculated the following metrics:

- Gross Margin = (revtq – cogsq) / revtq 
    - "revtq" represents total revenues and "cogsq" represents the Cost of Goods Sold both at quarter level. The difference equals gross profit.
- EBITDA = oiadpq + dpq
    - Earnings Before Interest, Tax, Depreciation and Amortization. Since oiadpq already deducts depreciation and amortisation, adding dpq back returns EBITDA.
- Net Debt = (dlcq + dlttq) – che
    - Net Debt measures a company’s overall debt situation by offsetting its total debt with its liquid assets.


#### a) Download data and save it as a .csv so we avoiding repeated long queries to WRDS in case we clear memory. 
If the data has already been downloaded do not run this and skip to b)

In [ ]:
# Define the directory to download into
directory = 'data/wrds_company_fundamentals_data'
output_file = os.path.join(directory, 'wrds_company_fundamentals_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Pass parameters to a sql statement
params = {
    "tickers": tuple(tickers),
    "start_date": earliest_date,
    "end_date": latest_date
}

# Query WRDS to fetch data
quartely_company_fundamentals_df = db.raw_sql(
    "SELECT rdq as date, tic as ticker, revtq as revenue, ceqq as book_value, niq as net_income, (revtq - cogsq) / NULLIF(revtq, 0) as gross_margin, (revtq - cogsq) / NULLIF(atq, 0) as gross_profitability, "
    "oiadpq / NULLIF(atq, 0) as operating_profitability, (dlcq + dlttq) / NULLIF(atq, 0) as leverage, dlcq + dlttq - cheq as net_debt, oiadpq + dpq as ebitda, "
    "(dlcq + dlttq - cheq) / NULLIF((oiadpq + dpq), 0) as netdebt_to_ebitda " 
    "FROM comp_na_daily_current.fundq " 
    "WHERE tic in %(tickers)s and rdq BETWEEN %(start_date)s AND %(end_date)s ",
    params=params
)
#  revtq as rev, cogsq as cogs, oiadpq as op_income, dlcq as st_debt, dlttq as lt_debt, cheq as cash_eq, atq as total_assets, niq as net_income,  - base fields. Keeping only calculated metrics.

# Save the DataFrame to the directory
quartely_company_fundamentals_df.to_csv(output_file, index=False)

#### b) Open .csv file to memory

In [ ]:
# Define the directory containing the CSV files
directory = 'data/wrds_company_fundamentals_data'

# Open the file in a DataFrame for viewing
quartely_company_fundamentals_file = os.path.join(directory, 'wrds_company_fundamentals_data.csv')
quartely_company_fundamentals_df = pd.read_csv(quartely_company_fundamentals_file)

Extract unique downloaded tickers and compare to unique ticker list derived so far. Delete unmatched tickers from previous datasets.

In [27]:
# Extract unique tickers from WRDS download
quartely_company_fundamentals_tickers = set(quartely_company_fundamentals_df['ticker'].unique().tolist())
print(f"{len(quartely_company_fundamentals_tickers)} unique tickers found in WRDS quarterly company fundamentals data")

# Find missing tickers (tickers in short_interest_df but NOT in quartely_company_fundamentals_df)
print(f"{len(tickers)} unique tickers found in FINRA short interest data")
missing_tickers = set(tickers) - quartely_company_fundamentals_tickers

print(f"{len(missing_tickers)} tickers missing from the WRDS quarterly company fundamentals compared to the short interest file")

# Remove records with missing tickers from short_interest_df and daily stock data
short_interest_df = short_interest_df[~short_interest_df['symbolCode'].isin(missing_tickers)]

# Update ticker variable
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"Updated ticker list. New unique ticker count is: {len(tickers)}")

6086 unique tickers found in WRDS quarterly company fundamentals data
16857 unique tickers found in FINRA short interest data
10771 tickers missing from the WRDS quarterly company fundamentals compared to the short interest file
Updated ticker list. New unique ticker count is: 6086


#### ii) Stock Data (Daily Level) - Prices and Volume
https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-daily/

Variable Reference:
- conm: company name
- datadate: record date
- tic: ticker symbol
- cshoc: shares outstanding
- cshtrd: trading Volume - daily
- prccd: price - close - daily

Filtering the data to fetch only USA stocks in order to be able to inspect the dataset sensibly without the need for currency conversions and consistency of financial statement data.

#### a) Download data and save it as a .csv so we avoiding repeated long queries to WRDS in case we clear memory. 
If the data has already been downloaded do not run this and skip to b)

In [ ]:
# Define the directory to download into
directory = 'data/wrds_stock_daily_data'
output_file = os.path.join(directory, 'wrds_stock_daily_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Pass parameters to a sql statement
params = {
    "tickers": tuple(tickers),
    "start_date": earliest_date,
    "end_date": latest_date
}

# Query WRDS to fetch data
daily_stock_data_df = db.raw_sql(
    "SELECT datadate, conm as company_name, tic as ticker, prccd as price_close, cshtrd as volume, cshoc as shares_outstanding, prccd * cshoc as market_cap, eps " 
    "FROM comp_na_daily_all.secd " 
    "WHERE tic in %(tickers)s and datadate BETWEEN %(start_date)s AND %(end_date)s AND fic = 'USA'",
    params=params
)

# Save the DataFrame to the directory
daily_stock_data_df.to_csv(output_file, index=False)

#### b) Open .csv file to memory

In [28]:
# Define the directory containing the CSV files
directory = 'data/wrds_stock_daily_data'

# Open the file in a DataFrame for viewing
daily_stock_data_file = os.path.join(directory, 'wrds_stock_daily_data.csv')
daily_stock_data_df = pd.read_csv(daily_stock_data_file)

Extract unique downloaded tickers and compare to short interest file dataset. Delete unmatched tickers from previous datasets.

In [29]:
# Extract unique tickers from WRDS download
daily_stock_data_tickers = set(daily_stock_data_df['ticker'].unique().tolist())
print(f"{len(daily_stock_data_tickers)} unique tickers found in WRDS daily stock data")

# Find missing tickers (tickers in ticker list but NOT in daily_stock_data_df)
print(f"{len(tickers)} unique tickers found in FINRA short interest data")
missing_tickers = set(tickers) - daily_stock_data_tickers

print(f"{len(missing_tickers)} tickers missing from the WRDS daily stock data compared to the short interest file")

# Remove records with missing tickers from short_interest_df and quartely_company_fundamentals_df
short_interest_df = short_interest_df[~short_interest_df['symbolCode'].isin(missing_tickers)]
quartely_company_fundamentals_df = quartely_company_fundamentals_df[~quartely_company_fundamentals_df['ticker'].isin(missing_tickers)]

# Update ticker variable
tickers = short_interest_df['symbolCode'].unique().tolist()
print(f"Updated ticker list. New unique ticker count is: {len(tickers)}")

4707 unique tickers found in WRDS daily stock data
6086 unique tickers found in FINRA short interest data
1379 tickers missing from the WRDS daily stock data compared to the short interest file
Updated ticker list. New unique ticker count is: 4707


### 3) Yahoo Finance Data

##### a) Market Data - S&P500 and VIX

In [ ]:
# Define the directory containing the CSV files
directory = 'data/yahoo_finance_sp500_vix_data'
output_file = os.path.join(directory, 'yahoo_finance_sp500_vix_data.csv')

# Check if the collated file already exists and delete it
if os.path.exists(output_file):
    os.remove(output_file)

# Define ticker symbols
tickers = ["^GSPC", "^VIX"]  # S&P 500 and VIX

# Fetch data (Closing Prices and Volume)
sp500_vix_data_df = yf.download(tickers, start=earliest_date, end=latest_date, progress=False)[['Close']]

# Flatten the MultiIndex to standard column names
sp500_vix_data_df.columns = [f"{col[0]}_{col[1]}" for col in sp500_vix_data_df.columns]

# Rename columns to match the requested format
sp500_vix_data_df = sp500_vix_data_df.rename(columns={
    'Close_^GSPC': 'sp500_price_close',
    'Close_^VIX': 'vix_price_close'
})

# Reorder columns
sp500_vix_data_df = sp500_vix_data_df[['sp500_price_close', 'vix_price_close']]

# Format date index to YYYY-MM-DD and reset index
sp500_vix_data_df.index = sp500_vix_data_df.index.strftime('%Y-%m-%d')
sp500_vix_data_df = sp500_vix_data_df.reset_index()

# Rename the date column to 'date'
sp500_vix_data_df = sp500_vix_data_df.rename(columns={'index': 'date'})

# Save the collated DataFrame to the same directory
sp500_vix_data_df.to_csv(output_file, index=False)


#### b) Open .csv file to memory

In [30]:
# Define the directory containing the CSV files
directory = 'data/yahoo_finance_sp500_vix_data'

# Open the collated file in a DataFrame for viewing
sp500_vix_data_file = os.path.join(directory, 'yahoo_finance_sp500_vix_data.csv')
sp500_vix_data_df = pd.read_csv(sp500_vix_data_file)

### 4) Join datasets and perform data cleansing

a) Join datasets and fowardfill datapoints where relevant

In [86]:
# Renaming columns used as join criteria between datasets
daily_stock_data_df.rename(columns={"datadate": "date"}, inplace=True)
sp500_vix_data_df.rename(columns={"Date": "date"}, inplace=True)
short_interest_df.rename(columns={"settlementDate": "date"}, inplace=True)
short_interest_df.rename(columns={"symbolCode": "ticker"}, inplace=True)

# Merge stock data and index data on date
merged_df = pd.merge(daily_stock_data_df, sp500_vix_data_df, on="date", how="left")

# Merge ticker-level data (on date + ticker)
merged_df = pd.merge(merged_df, quartely_company_fundamentals_df, on=["date", "ticker"], how="left")
merged_df = pd.merge(merged_df, short_interest_df, on=["date", "ticker"], how="left")

# Sort by date and ticker
merged_df = merged_df.sort_values(by=["date", "ticker"]).reset_index(drop=True)

# Forward fill ticker-specific, point-in-time values (e.g. short interest, quarterly gross profit, etc.)
ticker_cols = list(merged_df.columns)
ticker_cols.remove("date")  # Exclude date column from filling
ticker_cols.remove("ticker")  # Exclude ticker column from filling
merged_df[ticker_cols] = merged_df.groupby("ticker")[ticker_cols].ffill()



b) Cleanse dataset and improve readability

In [ ]:
# Scale relevant fields from short interest file and daily stock data to millions in order to match the fundamentals file
divisor = 1000000
cols_to_divide = ['volume', 'shares_outstanding', 'market_cap', 'short_volume', 'avg_daily_volume']

# Vectorized operation: divides the specified columns by the divisor.
merged_df[cols_to_divide] = merged_df[cols_to_divide] / divisor

# Step 1: Identify tickers with 0 shares outstanding.
tickers_zero_shares = merged_df.loc[merged_df['shares_outstanding'] == 0, 'ticker'].unique()
print("Count of tickers with 0 shares outstanding:", len(tickers_zero_shares))

# Remove rows corresponding to these tickers.
merged_df = merged_df[~merged_df['ticker'].isin(tickers_zero_shares)].copy()
print("DataFrame shape after removing tickers with 0 shares outstanding:", merged_df.shape)

# Step 2: Identify tickers with market_cap below $100,000,000.
tickers_low_market_cap = merged_df.loc[merged_df['market_cap'] < 100, 'ticker'].unique()
print("Count of tickers where market cap has been below 100,000,000:", len(tickers_low_market_cap))

# Remove rows corresponding to these tickers.
merged_df = merged_df[~merged_df['ticker'].isin(tickers_low_market_cap)].copy()
print("Final DataFrame shape after removing tickers with low market cap:", merged_df.shape)

# Step 3: Identify tickers where quarterly revenue been below $100,000.
tickers_low_revenue = merged_df.loc[merged_df['revenue'] < 0.1, 'ticker'].unique() 
print("Count of tickers with where quarterly revenue has been below $100,000:", len(tickers_low_revenue))

# Remove rows corresponding to these tickers.
merged_df = merged_df[~merged_df['ticker'].isin(tickers_low_revenue)].copy()
print("Final DataFrame shape after removing tickers with low revenue:", merged_df.shape)

# Final count of tickers
tickers_final = merged_df['ticker'].unique().tolist()
print(f"Final count of tickers is: {len(tickers_final)}")

# Switch column order for better readability
new_order = ['date', 'ticker', 'company_name', 'price_close', 'shares_outstanding', 'market_cap', 'volume', 'short_volume', 'avg_daily_volume', 'days_to_cover',
             'sp500_price_close', 'vix_price_close', 'eps', 'book_value', 'revenue', 'gross_margin', 'gross_profitability', 'ebitda', 'operating_profitability', 
             'net_income', 'netdebt_to_ebitrda', 'leverage'
            ],

merged_df = merged_df[new_order]

Count of tickers with 0 shares outstanding: 1
DataFrame shape after removing tickers with 0 shares outstanding: (3800015, 23)
Count of tickers where market cap has been below 100,000,000: 1609
Final DataFrame shape after removing tickers with low market cap: (2495330, 23)
Count of tickers with where quarterly revenue has been below $100,000: 206
Final DataFrame shape after removing tickers with low revenue: (2357504, 23)
Final count of tickers is: 2891


In [85]:
#merged_df.describe()
merged_df[merged_df['ticker'] == 'PTON'].sort_values(by=["date"]).head(10)
#len(merged_df['ticker'].unique().tolist())

,date,company_name,ticker,price_close,volume,shares_outstanding,market_cap,eps,sp500_price_close,vix_price_close,revenue,book_value,net_income,gross_margin,gross_profitability,operating_profitability,leverage,net_debt,ebitda,netdebt_to_ebitda,short_volume,avg_daily_volume,days_to_cover
1933257,2021-06-15,PELOTON INTERACTIVE INC,PTON,106.11,6.05,268.74,"28,516.43",0.91,"4,246.59",17.02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933258,2021-06-16,PELOTON INTERACTIVE INC,PTON,105.07,8.96,268.74,"28,236.93",0.91,"4,223.70",18.15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933259,2021-06-17,PELOTON INTERACTIVE INC,PTON,108.81,6.55,268.74,"29,242.03",0.91,"4,221.86",17.75,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933260,2021-06-18,PELOTON INTERACTIVE INC,PTON,109.12,6.36,268.74,"29,325.35",0.91,"4,166.45",20.70,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933261,2021-06-21,PELOTON INTERACTIVE INC,PTON,108.11,7.87,268.74,"29,053.91",0.91,"4,224.79",17.89,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933262,2021-06-22,PELOTON INTERACTIVE INC,PTON,117.17,13.08,268.74,"31,488.73",0.91,"4,246.44",16.66,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933263,2021-06-23,PELOTON INTERACTIVE INC,PTON,117.48,7.88,268.74,"31,572.05",0.91,"4,241.84",16.32,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933264,2021-06-24,PELOTON INTERACTIVE INC,PTON,121.85,12.75,268.74,"32,746.46",0.91,"4,266.49",15.97,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933265,2021-06-25,PELOTON INTERACTIVE INC,PTON,121.49,12.45,268.74,"32,649.71",0.91,"4,280.70",15.62,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06
1933266,2021-06-28,PELOTON INTERACTIVE INC,PTON,126.92,11.07,268.74,"34,108.99",0.91,"4,290.61",15.76,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,22.35,7.30,3.06


In [84]:
# Assume merged_df is already loaded.
# For example:
# merged_df = pd.read_csv('your_data.csv')

# Specify the field (column) you're interested in.
field_name = 'netdebt_to_ebitda'  # Replace with your actual column name

# Calculate the 99th percentile threshold for the chosen field.
threshold = merged_df[field_name].quantile(0.01)
print(f"99th percentile threshold for '{field_name}': {threshold}")

# Extract rows where the field's value is greater than or equal to the threshold.
filtered_df = merged_df[merged_df[field_name] < threshold]

# Sort the filtered DataFrame in descending order by the specified field.
sorted_filtered_df = filtered_df.sort_values(by=field_name, ascending=False)

# Display the full sorted DataFrame.
print("Filtered and sorted DataFrame:")
sorted_filtered_df.tail(10)


99th percentile threshold for 'netdebt_to_ebitda': -117.73723431281226
Filtered and sorted DataFrame:


,date,company_name,ticker,price_close,volume,shares_outstanding,market_cap,eps,sp500_price_close,vix_price_close,revenue,book_value,net_income,gross_margin,gross_profitability,operating_profitability,leverage,net_debt,ebitda,netdebt_to_ebitda,short_volume,avg_daily_volume,days_to_cover
3791161,2024-10-09,LIFE360 INC,LIF,42.07,0.10,74.14,"3,118.86",NaN,"5,792.04",20.86,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791135,2024-09-03,LIFE360 INC,LIF,37.25,0.23,74.14,"2,761.53",NaN,"5,528.93",20.72,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791160,2024-10-08,LIFE360 INC,LIF,40.95,0.15,74.14,"3,035.83",NaN,"5,751.13",21.42,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791159,2024-10-07,LIFE360 INC,LIF,41.58,0.23,74.14,"3,082.53",NaN,"5,695.94",22.64,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791138,2024-09-06,LIFE360 INC,LIF,37.50,0.36,74.14,"2,780.06",NaN,"5,408.42",22.38,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791158,2024-10-04,LIFE360 INC,LIF,40.02,0.21,74.14,"2,966.88",NaN,"5,751.07",19.21,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791147,2024-09-19,LIFE360 INC,LIF,35.70,0.28,74.14,"2,646.62",NaN,"5,713.64",16.33,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.17,0.37,1.00
3791134,2024-08-30,LIFE360 INC,LIF,39.43,0.23,74.14,"2,923.14",NaN,"5,648.40",15.00,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.18,0.33,1.00
3791154,2024-09-30,LIFE360 INC,LIF,39.35,0.14,74.14,"2,917.21",NaN,"5,762.48",16.73,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.35,0.30,1.18
3791151,2024-09-25,LIFE360 INC,LIF,39.05,0.18,74.14,"2,894.97",NaN,"5,722.26",15.41,84.86,326.72,-10.96,0.77,0.16,-0.01,0.00,-159.90,0.00,"-53,300.00",0.17,0.37,1.00
